# Exercise  

We have a layout file giving us the information on how we must interprete each file.

Files have their generation date in the file name with the format PYYMMDD.

We must create a class that takes each file, translates it and returns to us a DataFrame.

The dataframe columns must be:  
ID as portfolio_ID   
target ID as Amount   
operation date as trade date  
valuation date as settle date  
operation descritpion as desc (Internal Transaction)  
A column named estimation with values (E)  
We must include a column with the currency (MXN)  
And a column with a transaction_ID by concatenating three fields: portfolio_ID, Settle_date, operation descrition   

The final file must have all the processed transactions and have the filename generated with the next business day 
"cash_transactions_DATE.csv"

In [14]:
import pandas as pd

In [35]:
%%writefile layout_config.py
LAYOUT_MAPPING = {
    "ID":(1,3),
    "Price":(4,15),
    "OPERATION DATE":(16,23),
    "VALUATION DATE":(24,31),
    "TARGET VALUE":(32,46),
    "AMOUNT 1":(47,63),
    "AMOUNT 2":(64,80),
    "AMOUNT 3":(81,97),
    "AMOUNT 4":(98,114),
    "AMOUNT 5":(115,131),
    "AMOUNT 6":(132,148),
    "AMOUNT 7":(149,165)
}

Writing layout_config.py


In [44]:
from layout_config import LAYOUT_MAPPING

class ProcessFile():
    def __init__(self, filename):
        self._filename = filename
        
        self.clean_data()
    
    def clean_data(self):
    
        def _read_file():
            with open(self._filename, "r") as Pfile:
                self.string = Pfile.read()  
            
            return  
        
        def _get_layout():
            get_interval = lambda target_string, interval: target_string[interval[0] -1:interval[1]] 
            self.layout = {key:get_interval(self.string, interval) for key, interval in LAYOUT_MAPPING.items()}
            
            return
    
        def _process_layout():
            self.layout_df = pd.DataFrame.from_dict(self.layout, orient="index").T
            self.layout_df[["OPERATION DATE", "VALUATION DATE"]] = self.layout_df[["OPERATION DATE", "VALUATION DATE"]].apply(pd.to_datetime)

            return
        
        def _set_final_dataframe():
            pass
        
            return
        
        # main function(clean_data)
        _read_file()
        _get_layout()
        _process_layout()
        _set_final_dataframe()
        
        return
    

In [45]:
test_filename = r"C:\Users\2000076134\Dev\OneDrive_1_9-30-2022\P220602.955"
transaction_data = ProcessFile(test_filename)